In [1]:
from ipyleaflet import Map, Polygon, basemaps, basemap_to_tiles
import  numpy as np
from israel_coords import coords, green
import json
from ipyleaflet import GeoJSON
import random
from shapely.geometry import Polygon, LinearRing
from shapely.affinity import scale
israel = Polygon(np.array(coords).T)
# israel = scale(israel, xfact=1.1, yfact=1.1)
coords = np.stack(israel.exterior.xy).T
exterior = coords[:,::-1]
exterior = np.hstack((exterior[:-1],(exterior[:-1]+exterior[1:])/2)).reshape(-1,2)
exterior = np.hstack((exterior[:-1],(exterior[:-1]+exterior[1:])/2)).reshape(-1,2)
exterior = np.hstack((exterior[:-1],(exterior[:-1]+exterior[1:])/2)).reshape(-1,2)
exterior = np.hstack((exterior[:-1],(exterior[:-1]+exterior[1:])/2)).reshape(-1,2)
exterior = np.hstack((exterior[:-1],(exterior[:-1]+exterior[1:])/2)).reshape(-1,2)
exterior = np.hstack((exterior[:-1],(exterior[:-1]+exterior[1:])/2)).reshape(-1,2)
exterior = np.hstack((exterior[:-1],(exterior[:-1]+exterior[1:])/2)).reshape(-1,2)
exterior_upsample = exterior
exterior = coords[:, ::-1]
green = np.array(green)[:,::-1]
green = np.hstack((green[:-1],(green[:-1]+green[1:])/2)).reshape(-1,2)
green = np.hstack((green[:-1],(green[:-1]+green[1:])/2)).reshape(-1,2)
green = np.hstack((green[:-1],(green[:-1]+green[1:])/2)).reshape(-1,2)
green = np.hstack((green[:-1],(green[:-1]+green[1:])/2)).reshape(-1,2)
green = np.hstack((green[:-1],(green[:-1]+green[1:])/2)).reshape(-1,2)
green = np.hstack((green[:-1],(green[:-1]+green[1:])/2)).reshape(-1,2)
green = np.hstack((green[:-1],(green[:-1]+green[1:])/2)).reshape(-1,2)
green = np.hstack((green[:-1],(green[:-1]+green[1:])/2)).reshape(-1,2)
green = np.hstack((green[:-1],(green[:-1]+green[1:])/2)).reshape(-1,2)
green = np.hstack((green[:-1],(green[:-1]+green[1:])/2)).reshape(-1,2)



In [2]:
import json
from shapely.geometry import Point

with open("geocoded.json") as f:
    geocoded = json.load(f)

if 'nan' in geocoded:
    del geocoded['nan']
    
def first_contained(polygon, points):
    for p in points:
        p = (float(p['lat']),float(p['lon']))
        if polygon.contains(Point(*p[::-1])):
            return list(p)
    
address_to_coords = {k: first_contained(israel, v) for k,v in geocoded.items() if len(v)}
address_to_coords = {k: v for k,v in address_to_coords.items() if v is not None}

In [3]:
%matplotlib widget
from matplotlib import pyplot as plt
from scipy.spatial import voronoi_plot_2d, Voronoi
points = np.concatenate((np.array(list(address_to_coords.values())), exterior_upsample, green))
color = np.concatenate((np.ones(len(address_to_coords.values())),np.ones(len(exterior_upsample))*2, np.ones(len(green))*3))
vor = Voronoi(points)

In [4]:

def sub_poly(coords,props):
    return {"geometry": {
                  "type": "Polygon", 
                  "coordinates": [coords[::-1]]
              }, 
      "type": "Feature", 
      "properties": props
      }
addresses = list(address_to_coords.keys())
index_of_regions = vor.point_region[color==1]
features = []
regional_points ={
    "type": "FeatureCollection", 
    "features": [
      
    ]
}
for i, region_index in enumerate(index_of_regions):
    region = Polygon([vor.vertices[v].tolist() for v in vor.regions[region_index]]).exterior.xy
    region = np.stack(region).T[:,::-1].tolist()
    properties = {"address": addresses[i][:-1]}
    features.append(sub_poly(region, properties))

regional_points["features"] = features

In [5]:
import numpy as np
import pandas as pd

df = pd.read_csv("test.csv")
city_column = 'שם ישוב בחירות'
address_column = 'כתובת קלפי'
building_number_column = 'מקום קלפי'
address_tokenized_city_column = 'סמל ישוב בחירות'
address_kalpi_token_column = 'סמל קלפי'
df['address'] = (df[address_column] + " " + df[building_number_column] + "," + df[city_column] + ", ישראל")

votes_df = pd.read_csv("expb.csv")
votes_tokenized_city_column = 'סמל ישוב'
votess_kalpi_token_column = 'קלפי'

def get_address_area(df, address):
    return df[df['address'] == address]

def get_area_votes(votes_df, area):
    city_votes = votes_df[votes_df[votes_tokenized_city_column].isin(area[address_tokenized_city_column])]
    address_votes = city_votes[city_votes[votess_kalpi_token_column].isin(area[address_kalpi_token_column])]
    return address_votes

def summarize_votes(df):
    votes = df[df.columns[8:11]]
    votes_by_party = df[df.columns[11:]]
    return votes.sum(), votes_by_party.sum()

def summarize_address(df, votes, address):
    area = get_address_area(df, address)
    votes = get_area_votes(votes, area)
    return summarize_votes(votes)


In [6]:
from matplotlib import pyplot as plt
from matplotlib.ticker import PercentFormatter
plt.ioff()
fig, party_ax = plt.subplots(1,1)
plt.ion()

def plot_party_votes_df(party_votes, ax):    
    ax.cla()
    plot_data= party_votes.sort_values().copy()
    plot_data = plot_data * 100 /plot_data.sum()
    plot_data.index = map(lambda s: s[::-1],plot_data.index)
    plot_data.plot(kind='barh',ax=ax)
    ax.set_ylim([len(plot_data)-10,len(plot_data)])
    ax.set_xlim([0,100])
    ax.xaxis.set_major_formatter(PercentFormatter())
    
def plot_general_voting_df(general_votes, ax):
    ax.cla()
    plot_data= general_votes.sort_values().copy()
    plot_data = plot_data[:2] * 100 /plot_data[2]
    plot_data.index = map(lambda s: s[::-1],plot_data.index)
    plot_data.plot(kind='bar',ax=ax, color=['red', 'green'])
    
def plot_address_summary(df, votes, address,  party_ax):
    general, party = summarize_address(df, votes, address)
    plot_party_votes_df(party, party_ax)
    party_ax.figure.suptitle(address[::-1])

In [7]:
from ipywidgets import Label, VBox, FloatProgress, HTML
address = 'כצנלסון ישראל 10 ,רחובות, ישראל'


title_widget = HTML()

def set_format(fmt):
    def set_in_fmt(widget,variable):
        widget.value = fmt % variable
    return set_in_fmt

set_title = set_format("<h1>%s</h1>")
set_voted = set_format('מצביעים: %s')
set_invalid = set_format('פסולים: %s, %.2f%%')
set_valid = set_format('כשרים: %s, %.2f%%')
set_dominant_party = set_format('מפלגה דומיננטית: %s')
set_dominant_party_votes = set_format('הצבעות למלגה הדומיננטית: %s, %.2f%%')

voted_label = Label(style={"font_size":"23px"})
invalid_label = Label(style={"font_size":"23px"})
valid_label = Label(style={"font_size":"23px"})
dominant_party_label = Label(style={"font_size":"23px", "font_weight": "bold"})
dominant_party_votes_label = Label(style={"font_size":"23px"})

set_voted(voted_label, '')
set_valid(invalid_label, ('',0))
set_invalid(valid_label, ('',0))
set_title(title_widget, "Hover to display quick summary")
set_dominant_party(dominant_party_label, '')
set_dominant_party_votes(dominant_party_votes_label, ('', 0))

def get_dominant_party(party):
    i = party.argmax()
    return party.index[i],party[i],party[i]/party.sum()

def update_summary_labels(address, df, votes_df):
    general, party = summarize_address(df, votes_df,address)
    set_title(title_widget, address)
    set_voted(voted_label, general[2])
    set_valid(invalid_label, (general[2],general[2]*100/general[0]))
    set_invalid(valid_label, (general[1],general[1]*200/general[0]))
    name, votes, percent = get_dominant_party(party)
    set_dominant_party(dominant_party_label, name)
    set_dominant_party_votes(dominant_party_votes_label, (votes, percent))
    
labels = VBox([title_widget, dominant_party_label, voted_label, valid_label, invalid_label, dominant_party_votes_label])

In [8]:
from ipyleaflet import Map, Polygon as PolygonLayer, basemaps, basemap_to_tiles, FullScreenControl, WidgetControl
p = PolygonLayer(locations=exterior.tolist(), color="gray", opacity=0.001) 
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(31.5,35),
    zoom=8,
    layout = {
        'height': "100vh"
    },
    scroll_wheel_zoom=True
)

def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

geo_json = GeoJSON(
    data=regional_points,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)
geo_json.on_hover(lambda feature, **kwargs: update_summary_labels(feature['properties']['address'], df, votes_df))
geo_json.on_click(lambda feature, **kwargs: plot_address_summary(df, votes_df, feature['properties']['address'], party_ax=party_ax))
m.add_layer(p)
m.add_layer(geo_json)
m.add_control(FullScreenControl())
m.add_control(WidgetControl(widget=VBox([labels, fig.canvas], layout={"height": "100vh", "width": "30vw"}), position="topright"))

In [9]:
m

Map(center=[31.5, 35], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…